In [2]:
import pandas as pd
import numpy as np

In [27]:
df_sale = pd.read_csv('/Users/ruianyang/Documents/GradSchool/MSDS/DATA515/group_project/Extr_RPSale.csv', 
                      dtype={'Mayor': str, 'Minor':'str'})
df_building = pd.read_csv('/Users/ruianyang/Documents/GradSchool/MSDS/DATA515/group_project/EXTR_ResBldg.csv')
df_parcel = pd.read_csv('/Users/ruianyang/Documents/GradSchool/MSDS/DATA515/group_project/EXTR_Parcel.csv',
                       encoding = "ISO-8859-1")
df_lookup = pd.read_csv('/Users/ruianyang/Documents/GradSchool/MSDS/DATA515/group_project/EXTR_LookUp.csv')

/Users/ruianyang/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ruianyang/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
df_lookup_items = pd.read_csv('/Users/ruianyang/Documents/GradSchool/MSDS/DATA515/group_project/look_up_item.csv')
df_col_names = pd.read_csv('/Users/ruianyang/Documents/GradSchool/MSDS/DATA515/group_project/column_names.csv')

In [29]:
df_sale.columns = df_col_names[df_col_names['source'] == 'sale'].name.tolist()
df_building.columns = df_col_names[df_col_names['source'] == 'building'].name.tolist()
df_parcel.columns = df_col_names[df_col_names['source'] == 'parcel'].name.tolist()
df_lookup.columns = df_col_names[df_col_names['source'] == 'lookup'].name.tolist()

In [30]:
df_lookup['Look Up Description'] = df_lookup['Look Up Description'].str.strip()

In [34]:
df_parcel_sf = df_parcel[df_parcel['Property Type'] == 'R']

In [37]:
df_parcel_sf = df_parcel_sf.drop(columns=['Property Type'])

In [66]:
df_sale_sf = df_sale[df_sale['Property Type'] == 11]

In [33]:
df_building_sf = df_building[df_building['Number Living Units'] == 1]

In [67]:
df_sale_sf['Document Date'] = pd.to_datetime(df_sale_sf['Document Date'])

/Users/ruianyang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
df_sale_sf_recent = df_sale_sf[df_sale_sf['Document Date'] > '2010-01-01']

In [69]:
new_df = pd.merge(df_sale_sf_recent, df_building_sf,  how='left', left_on=['Major','Minor'], right_on=['Major','Minor'])
df_all = pd.merge(new_df, df_parcel_sf,  how='left', left_on=['Major','Minor'], right_on=['Major','Minor'])

In [72]:
for col in df_all.columns:
    #print(col)
    if col in df_lookup_items['Field Name'].tolist():
        look_up_type = int(df_lookup_items[df_lookup_items['Field Name'] == col]['Look Up'])
        look_up_items = df_lookup[df_lookup['Look Up Type'] == look_up_type]
        print(col)
        description_list = []
        for i in range(len(df_all[col])):
            num = df_all[col].iloc[i]
            #print(num)
            description = look_up_items[look_up_items['Look Up Item'] == num]['Look Up Description']
            #print(description.values)
            if len(description) == 0:
                description_list.append('nan')
            else:
                description_list.append(description.values[0])
                #print(description.values[0])
        df_all[col] = description_list
                

Property Type
Principal Use
Sale Instrument
Sale Reason
Property Class
Sale Warning


/Users/ruianyang/anaconda/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


HBU As If Vacant
HBU As Improved
Present Use
Water System
Sewer System
Access
Topography
Street Surface
Restrictive Size / Shape
Inadequate Parking
View: Mt Rainier
View: Olympics
View: Cascades
View: Territorial
View: Seattle Skyline
View: Puget Sound
View: Lake Washington
View: Lake Sammamish
View: Small Lake / River / Creek
View: Other View
Waterfront Location
Waterfront Bank
Waterfront Poor Quality
Waterfront Restricted Access
Tideland / Shoreland
Traffic Noise
Contamination
Common Property
Historic Site
Current Use Designation


In [79]:
df_all.to_csv('/Users/ruianyang/Documents/GradSchool/MSDS/DATA515/group_project/merged_kc_data.csv')